In [ ]:
import pandas as pd
import ast
import re

In [ ]:
# Read the chordonomicon dataset
df = pd.read_csv('chordonomicon.csv')

# Read the mapping CSV file
chord_relations = pd.read_csv('chords_mapping.csv')
# Create a dictionary with keys the "chords" and values the "degrees"
chord_degrees = dict(zip(chord_relations['Chords'], chord_relations['Degrees']))
for key, value in chord_degrees.items():
    chord_degrees[key] = ast.literal_eval(value)
chord_notes = dict(zip(chord_relations['Chords'], chord_relations['Notes']))
for key, value in chord_notes.items():
    chord_notes[key] = ast.literal_eval(value)


In [ ]:
# remove inversions
df['chords'] = df['chords'].apply(lambda s: re.sub(r"/[^/]*$", "", s))


In [ ]:
# map the chords to notes and binary degree table
for progression in df['chords']:
    for chord in progression.split():
            if ">" not in chord:
                notes = chord_notes[chord]
                degrees = chord_degrees[chord]

In [24]:
# function for pitch shifting
degrees_sharp = ['C', 'Cs', 'D', 'Ds', 'E', 'F', 'Fs', 'G', 'Gs', 'A', 'As', 'B']
degrees_flat = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']


#def convert_chord(chord, shift):
def convert_chord(chord, shift):
    converted_chord = []
    converted_chord_base = []
    chord_progression = chord.split(' ')
    for cchord in chord_progression:
        if '>' in cchord:
            converted_chord.append(cchord)
            converted_chord_base.append(cchord)
        elif cchord == '':
            pass
        else:
            degree = 'next'
            rest = 'next'
            for note in all_notes_list:
                if cchord.startswith(note):
                    if 'sus' in cchord and 'ssus' not in cchord and 'bsus' not in cchord and '7sus' not in cchord:
                        break
                    else:
                        degree = note
                        degree = degree.replace("Bs",'C').replace('Cb',"B").replace('Es','F').replace('Fb','E')
                        rest = cchord.replace(note, '', 1)
                        break
                else:
                    for note in notes:
                        if cchord.startswith(note):
                            degree = note
                            degree = degree.replace("Bs",'C').replace('Cb',"B").replace('Es','F').replace('Fb','E')
                            rest = cchord.replace(note, '', 1)
                            break

            if degree in degrees_sharp:
                index = degrees_sharp.index(degree)
                next_index = (index + shift) % len(degrees_sharp)
                next_degree = degrees_sharp[next_index]
            elif degree in degrees_flat:
                index = degrees_flat.index(degree)
                next_index = (index + shift) % len(degrees_flat)
                next_degree = degrees_flat[next_index]
            else:
                next_degree = 'next'
                

            next_base = ''  # Initialize next_base here

            if '/' in cchord:
                base = cchord.split('/')[1]
                extension = rest.replace(base, '').replace('/', '')
                if base in degrees_sharp:
                    index = degrees_sharp.index(base)
                    next_index = (index + shift) % len(degrees_sharp)
                    next_base = degrees_sharp[next_index]
                elif base in degrees_flat:
                    index = degrees_flat.index(base)
                    next_index = (index + shift) % len(degrees_flat)
                    next_base = degrees_flat[next_index]

                next_base = next_base.replace("Bs",'C').replace('Cb',"B").replace('Es','F').replace('Fb','E')
                converted_chord_base.append(next_degree + extension + '/' + next_base)
                converted_chord.append(next_degree + extension)
                
            else:
                converted_chord.append(next_degree + rest)
                converted_chord_base.append(next_degree + rest)
    return converted_chord, converted_chord_base



notes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
accs = ['b', 's']
all_notes_list = [note + acc for note in notes for acc in accs]

In [25]:

for progression in df['chords'][:2]:
    # shift = 5
    converted_progression, converted_progression_base = convert_chord(progression, shift)

<intro_1> C <verse_1> F C E7 Amin C F C G7 C F C E7 Amin C F G7 C <verse_2> F C E7 Amin C F C G7 C F C E7 Amin C F G7 C <chorus_1> F C F C G C F C E7 Amin C F G7 C <solo_1> D <chorus_2> G D G D A D G D Fs7 Bmin D G A7 D G A7 D
['<intro_1>', 'F', '<verse_1>', 'As', 'F', 'A7', 'Dmin', 'F', 'As', 'F', 'C7', 'F', 'As', 'F', 'A7', 'Dmin', 'F', 'As', 'C7', 'F', '<verse_2>', 'As', 'F', 'A7', 'Dmin', 'F', 'As', 'F', 'C7', 'F', 'As', 'F', 'A7', 'Dmin', 'F', 'As', 'C7', 'F', '<chorus_1>', 'As', 'F', 'As', 'F', 'C', 'F', 'As', 'F', 'A7', 'Dmin', 'F', 'As', 'C7', 'F', '<solo_1>', 'G', '<chorus_2>', 'C', 'G', 'C', 'G', 'D', 'G', 'C', 'G', 'B7', 'Emin', 'G', 'C', 'D7', 'G', 'C', 'D7', 'G']
<intro_1> E D A E D A <verse_1> E D A E D A E D A E D A C <chorus_1> E G D A E G D A E G D A C D E D A <verse_2> E D A E D A E D A E D A C <chorus_2> E G D A E G D A E G D A C D <bridge_1> E C G D E C G D E C G D C D E G E G D A E G D A E G D A C D <chorus_3> E G D A E G D A E G D A C D <bridge_2> E C G D E C G D 

In [22]:
converted_progression

['nextnext',
 'Fmaj7',
 'F',
 'Fmaj7',
 'F',
 'nextnext',
 'C',
 'G',
 'Amin',
 'G',
 'Amin',
 'Dmin',
 'G',
 'C',
 'nextnext',
 'F',
 'Fmaj7',
 'F',
 'C',
 'Fmaj7',
 'F',
 'Dmin7',
 'Amin',
 'nextnext',
 'C',
 'Fmaj7',
 'F',
 'Dmin7',
 'nextnext',
 'C',
 'F',
 'C',
 'F',
 'Dmin7',
 'C',
 'F',
 'Dmin7',
 'G',
 'nextnext',
 'C',
 'F',
 'Dmin',
 'F',
 'nextnext',
 'C',
 'F',
 'C',
 'F',
 'Dmin',
 'C',
 'nextnext',
 'Fmaj7',
 'F',
 'Dmin']